# Scrape the LangChain documentation into a ChromaDB Vector Database and use it for a GPT-4 chatbot  to talk with it!

In this notebook, I will introduce you to vector databases. I will:
- Web scrape the LangChain documentation
- Store the LangChain documentation in a Chroma DB vector database
- Create a retriever to retrieve the desired information
- Create a Q&A chatbot with GPT-4
- Show how you can delete and reopen a vector database locally to save space
Visualise your vector database (very cool, read till the end!)

This notebook is connected to a medium article: [Medium articles](https://medium.com/@rubentak)

ref: https://medium.com/@rubentak/unleashing-the-power-of-intelligent-chatbots-with-gpt-4-and-vector-databases-a-step-by-step-8027e2ce9e78

In [63]:
!pip install selenium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 9.3 MB/s eta 0:00:0000:01

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [72]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extract_urls_with_js(url):
    options = Options()
    options.headless = True  # Run Chrome in headless mode

    # Provide the path to your ChromeDriver executable
    driver_path = "/home/zhuoli/Projects/github/zhuoli/data/chromedriver"

    # Initialize Chrome WebDriver
    driver = webdriver.Chrome(driver_path)

    try:
        driver.get(url)
        
        # Wait for the dynamic content to load (you might need to adjust the wait time)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.ID, "app")))

        # Get the page source after JavaScript execution
        page_source = driver.page_source
    finally:
        driver.quit()

    # Now, parse the page source with BeautifulSoup as before
    soup = BeautifulSoup(page_source, 'html.parser')
    urls = set()

    for link in soup.find_all('a', href=True):
        absolute_url = urljoin(url, link['href'])
        urls.add(absolute_url)

    return urls



In [73]:
target_url = "https://docs.oracle.com/en-us/iaas/api/#/"  # Replace with the URL you want to scrape
found_urls = extract_urls_with_js(target_url)

print("Found URLs:")
for url in found_urls:
    print(url)

/tmp/ipykernel_28439/3022003982.py:11: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True  # Run Chrome in headless mode


AttributeError: 'str' object has no attribute 'capabilities'

In [34]:
import os
def save_content(link_list):
    for i, link in enumerate(link_list):
        html_data = get_data(link)
        soup = BeautifulSoup(html_data, "html.parser")
        text = soup.get_text()

        # Remove the first 835 lines
        lines = text.splitlines()
        cleaned_text = "\n".join(lines)

        # Get the first 3 words in the cleaned text
        words = cleaned_text.split()[:3]
        file_name_prefix = "_".join(words)

        # Replace special characters and spaces with an underscore
        file_name_prefix = re.sub(r"[^a-zA-Z0-9]+", "_", file_name_prefix)

        # Get the current working directory
        current_dir = os.getcwd()

        # Move up one level to the parent directory
        parent_dir = os.path.dirname(current_dir)

        # Set the path to the data folder
        data_folder = os.path.join(parent_dir, "data/langchain_doc")

        # Create the data folder if it doesn't exist
        if not os.path.exists(data_folder):
            os.makedirs(data_folder)

        # Set the path to the output file
        output_file = os.path.join(data_folder, f"{i}_{file_name_prefix}.txt")

        # Save the cleaned content to the output file
        with open(output_file, "w") as f:
            f.write(cleaned_text)

In [35]:
# save the content of the links into txt files
save_content(found_urls)

# Q&A bot with langchain over a directory

In [36]:
# Import libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader

In [49]:
# Create a new openai api key
import os

api_key = os.environ.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = "sk-Your key"
# set up openai api key
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [40]:
# Print number of txt files in directory
loader = DirectoryLoader('/home/zhuoli/Projects/github/zhuoli/data/langchain_doc', glob="./*.txt")
doc = loader.load ( )
len(doc)

[nltk_data] Downloading package punkt to /home/zhuoli/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/zhuoli/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


16

In [41]:
# Splitting the text into chunks
text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(doc)

In [42]:
# Count the number of chunks
len(texts)

76

In [43]:
# Print the first chunk
texts[0]

Document(page_content="Cloud Free Tier | Oracle\n\nAccessibility Policy\n\nSkip to content\n\nAbout\n\nServices\n\nSolutions\n\nPricing\n\nPartners\n\nResources\n\nClose Search\n\nClose\n\nWe’re sorry. We could not find a match for your search. We suggest you try the following to help find what you're looking for:\n\nCheck the spelling of your keyword search. Use synonyms for the keyword you typed, for example, try “application” instead of “software.” Start a new search.\n\nClear Search\n\nSearch\n\nMenu\n\nMenu\n\nContact Sales\n\nSign in to Oracle Cloud\n\nCloud\n\nOracle Cloud Free Tier Build, test, and deploy applications on Oracle Cloud—for free. New Always Free services have been added, including Arm Ampere A1 Compute. For large-scale Arm development projects you can apply for  OCI Arm Accelerator.\n\nStart for free Sign in to Oracle Cloud\n\nOracle Cloud Infrastructure Always Free Services (0:38)\n\nWhat's included with Oracle Cloud Free Tier? *\n\nAlways Free services Services 

In [47]:
!pip install openai
!pip install tiktoken


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
  Using cached tiktoken-0.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


# Data base creation with ChromaDB

https://www.youtube.com/watch?v=3yPBVii7Ct0

In [50]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

# OpenAI embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [51]:
# Persist the db to disk
vectordb.persist()
vectordb = None

In [52]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

# Create retriever

In [53]:
retriever = vectordb.as_retriever()

In [56]:
docs = retriever.get_relevant_documents("write a resignation letter for me, my last day is this Friday, and I'm happy to leave this shithole")

In [57]:
docs

[Document(page_content='Moved', metadata={'source': '/home/zhuoli/Projects/github/zhuoli/data/langchain_doc/2_Moved.txt'}),
 Document(page_content='Moved', metadata={'source': '/home/zhuoli/Projects/github/zhuoli/data/langchain_doc/12_Moved.txt'}),
 Document(page_content='Submit\n\nPlease complete this form and a sales representative will contact you. (change)\n\nI’m an Oracle customer who needs support Get technical, billing, and account questions answered.\n\nI need technical support\n\nFind the\n\ncustomer support\n\nyou need.\n\nUse the following information to get technical help:', metadata={'source': '/home/zhuoli/Projects/github/zhuoli/data/langchain_doc/15_Oracle_Contacts_Click.txt'}),
 Document(page_content='Please select product category\n\nCloud Applications\n\nCloud Infrastructure\n\nDatabase\n\nJava\n\nLinux\n\nIndustry Solutions\n\nOn\n\n\n\nPremise Products\n\nProduct\n\nPlease select product Sales, Marketing, and Commerce Service and Field Service Financial (ERP/EPM/Pro

In [44]:
len(docs)

4

In [58]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [46]:
retriever.search_type

'similarity'

In [47]:
retriever.search_kwargs

{'k': 2}

# Create a question answering chain

In [59]:
# Create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True,
                                  verbose=True)

In [60]:
# Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [61]:
# Question
query = "write a resignation letter for me, my last day is this Friday, and I'm happy to leave this shithole"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new RetrievalQA chain...

> Finished chain.
 I am writing to formally resign from my current position, effective this Friday. It has been a pleasure working here, but I am ready to move on to something new. Thank you for the opportunity.


Sources:
/home/zhuoli/Projects/github/zhuoli/data/langchain_doc/2_Moved.txt
/home/zhuoli/Projects/github/zhuoli/data/langchain_doc/12_Moved.txt


In [ ]:
# Break it down
query = "What are all agent types?"
llm_response = qa_chain(query)
process_llm_response(llm_response)
#llm_response

qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [55]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


# Deleteing the DB

In [250]:
!zip -r db.zip ./db

updating: db/ (stored 0%)
updating: db/chroma-embeddings.parquet (deflated 29%)
updating: db/index/ (stored 0%)
updating: db/index/index_metadata_b9a5e02f-ebd0-4b13-8858-b30b211c4546.pkl (deflated 5%)
updating: db/index/id_to_uuid_b9a5e02f-ebd0-4b13-8858-b30b211c4546.pkl (deflated 37%)
updating: db/index/uuid_to_id_d80886e4-65e1-4231-8c73-99ff58d68061.pkl (deflated 39%)
updating: db/index/index_b9a5e02f-ebd0-4b13-8858-b30b211c4546.bin (deflated 17%)
updating: db/index/index_d80886e4-65e1-4231-8c73-99ff58d68061.bin (deflated 17%)
updating: db/index/uuid_to_id_b9a5e02f-ebd0-4b13-8858-b30b211c4546.pkl (deflated 41%)
updating: db/index/id_to_uuid_d80886e4-65e1-4231-8c73-99ff58d68061.pkl (deflated 32%)
updating: db/index/index_metadata_d80886e4-65e1-4231-8c73-99ff58d68061.pkl (deflated 5%)
updating: db/chroma-collections.parquet (deflated 50%)
updating: db/.DS_Store (deflated 96%)


In [251]:
# To clean up, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# Delete the directory
!rm -rf db/

# Starting again loading the db

In [57]:
!unzip db.zip

Archive:  db.zip
replace db/chroma-embeddings.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-..."

In [59]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

Using embedded DuckDB with persistence: data will be stored in: db


#### Usung turbo GPT API

In [60]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [61]:
# Create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True,
                                  verbose=True)

In [62]:
# Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [63]:
# Question
query = "What are the agent types?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new RetrievalQA chain...

> Finished chain.
There are two main types of agents mentioned in the context: Action Agents and Plan-and-Execute Agents. Action Agents decide the actions to take and execute those actions one at a time, while Plan-and-Execute Agents first decide a plan of actions to take, and then execute those actions one at a time.


Sources:
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/639_Agents_Contents_Action.txt
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/344_Agents_Contents_Action.txt


In [64]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [65]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}


# Visualizing the Vector db
https://github.com/mtybadger/chromaviz?ref=reactjsexample.com

https://github.com/avantrio/chroma-viewer


In [ ]:
from chromaviz import visualize_collection
visualize_collection(vectordb._collection)